Check out the YouTube video that explains this topic: https://youtu.be/CG3EV7UBELA

In [1]:
import pandas as pd
import numpy as np


Create a dataset with 10,000 rows of random values between 0 and 1

In [2]:
df = pd.DataFrame(np.random.rand(10000,5), columns=('A','B','C','D','E'))

In [3]:
df.describe()

,A,B,C,D,E
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.496377,0.498247,0.497853,0.502078,0.500196
std,0.289925,0.287222,0.289450,0.290236,0.287587
min,0.000314,0.000015,0.000019,0.000008,0.000037
25%,0.246579,0.248700,0.245901,0.250680,0.248988
50%,0.493952,0.502909,0.498671,0.503460,0.503903
75%,0.744602,0.742103,0.749529,0.752737,0.749442
max,0.999714,0.999938,0.999286,0.999943,0.999978


## **iterrows**

In [4]:
for i, row in df.iterrows():
    while i < 3:
        print(f'i = {i}')
        print('')
        print('Returned Row:')
        print(row)
        print('')
        print(f"Column B: {row['B']}")
        print("<----------------------------------->")
        print('')
        break

i = 0

Returned Row:
A    0.250116
B    0.128666
C    0.673821
D    0.964676
E    0.673199
Name: 0, dtype: float64

Column B: 0.1286656224205216
<----------------------------------->

i = 1

Returned Row:
A    0.468977
B    0.418127
C    0.182244
D    0.608990
E    0.240369
Name: 1, dtype: float64

Column B: 0.4181273810093029
<----------------------------------->

i = 2

Returned Row:
A    0.441664
B    0.654409
C    0.398197
D    0.691576
E    0.852308
Name: 2, dtype: float64

Column B: 0.6544087598653868
<----------------------------------->




If we compare the first three rows returned with df.iterrows to the first three rows from df.head(3) we see they are the same :)



In [5]:
df.head(3)

,A,B,C,D,E
0,0.250116,0.128666,0.673821,0.964676,0.673199
1,0.468977,0.418127,0.182244,0.608990,0.240369
2,0.441664,0.654409,0.398197,0.691576,0.852308


In [7]:
# .at[i,col] means at index col

def iterrow_example(df, col):
    for i, row in df.iterrows():
        val = row[col]
        if val < .5:
            df.at[i,col] = 0     
        else:
            df.at[i,col] = 1

Here we use the magic method timeit to let us know how long it takes to execute the line of code
We allso call the iterrow_example function
In this example I got 553 ms ± 8.24 ms (If you execute this code it may be different for you, but the patterns bellow should hold)

In [8]:
%timeit iterrow_example(df, 'A')

1 loop, best of 3: 685 ms per loop


## **pd.loc / pd.iloc**

Another way we can loop over the rows of a df is to use pd.loc or pd.iloc.
We create a function below to iterate over the df and then we call it.

In [10]:
def iloc_test(df, col):
    for i in df.index:
        val = df[col].iloc[i]
        if val < .5:
            df.at[i,col] = 0     
        else:
            df.at[i,col] = 1

In [11]:
%timeit iloc_test(df, 'B')

1 loop, best of 3: 199 ms per loop


## **Apply with lambda**

Here we use the .apply method with a lambda function to get even better performance.
In this example I got 3.51 ms ± 216 µs.
That's over 50 times faster than our previous best with pd.iloc!
LET'S GO FASTER!!!

In [12]:

%timeit df['C'] = df['C'].apply(lambda x : 0 if x < .5 else 1)

100 loops, best of 3: 3.94 ms per loop


## **np.where()**


https://www.youtube.com/watch?v=nxWginnBklU

np.where( conditional statment -> bool array, series/array/function()/scalar if true, series/array/function()/scalar if False )

Here we use the np.where method where we pass in our entire column of data with the condition we are testing for. Then we pass in the value we want if the condition is true and the value if the condition is false.

223 µs ± 13.4 µs! 16 times faster than our previous best with .apply()
Can we go any faster? Yup!

In [13]:
%timeit df['D'] = np.where(df['D'] < .5, 0, 1)

The slowest run took 99.05 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 411 µs per loop



When can squeeze out a little more performance by converting our pandas series into a numpy array by using .values

93.2 µs ± 527 ns
Over 2 times speed up from our previous best and almost 6,000 times faster than df.iterrows.
These numbers are relative though. With a smaller amount of data the differences wouldn't be as big, but with larger datasets the differences would be even more exaggerated.

In [14]:
%timeit df['E'] = np.where(df['E'].values < .5, 0, 1)

The slowest run took 11.16 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 177 µs per loop


If you have elif conditions thne use numpy.select instead.
conditions can be chained ((df[col ==no]) & (df['Providers']==0))

In [ ]:
conditions = [] # List all the conditions 
choices = [] # List all of the options
df[col] = np.select(conditions, choices, default='NA')

## **References**



YouTube Videos (Pycon Talks)
1000x faster data manipulation: vectorizing with Pandas and Numpy
https://www.youtube.com/watch?v=nxWginnBklU

Sofia Heisler No more sad pandas optimizing pandas code for speed and efficiency PyCon 2017
https://www.youtube.com/watch?v=HN5d490_KKk&feature=youtu.be


Articles
Optimum approach for iterating over a DataFrame
https://medium.com/@rtjeannier/pandas-101-cont-9d061cb73bfc

Crude looping in Pandas, or That Thing You Should Never Ever Do
https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6